<font color=gray>ADS Sample Notebook.

Copyright (c) 2021 Oracle, Inc. All rights reserved. Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***

# `ADSTuner` - Update the Search Space
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Team </font></p>

***

## Overview:

This notebook demonstrates how to perform a hyperparameter tuning on a toy model. It explains how to adjust the search space used by the `ADSTuner`. It illustrates how to add and remove hyperparameters. Also, the notebook shows you the techniques that are needed to adjust the parameters that define the exact distributions used to define the search space. 

**Note:** The `adstuner.ipynb` notebook contains an overview of the `ADSTuner` class.

---

## Prerequisites:
- Experience with the topic: Intermediate
- Professional experience: Intermediate

---

## Objectives:


- <a href='#intro'>Introduction</a>
- <a href='#create'>Create an `ADSTuner` Object</a>
- <a href='#modify'>Modify the Search Space</a>
  - <a href='#modify_update'>Update Distribution Hyperparameters</a>
  - <a href='#modify_remove'>Remove Distribution Hyperparameters</a>
  - <a href='#modify_add'>Add Distribution Hyperparameters</a>
- <a href='#ref'>References</a>
---

**Important:**

Placeholder text for required values are surrounded by angle brackets that must be removed when adding the indicated content. For example, when adding a database name to `database_name = "<database_name>"` would become `database_name = "production"`.

---

<font color=gray>Datasets are provided as a convenience.  Datasets are considered third party content and are not considered materials under your agreement with Oracle applicable to the services. The `iris` dataset is distributed under the [BSD license](https://github.com/scikit-learn/scikit-learn/blob/master/COPYING).</font>

In [ ]:
from ads.hpo.search_cv import ADSTuner
from ads.hpo.stopping_criterion import *
from ads.hpo.distributions import *
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.ensemble import GradientBoostingClassifier

<a id='intro'></a>
# Introduction

When performing a hyperparameter tuning operation, you have to decide which parameters to tune and those parameters to hold constant. For the hyperparameters that are being tuned, you have to select a sampling distribution family for each one. Examples of the sampling distribution families are uniform, log-uniform, discrete uniform, and categorical. Each sampling distribution family accepts parameters that define the exact sampling distribution. You define the search space with what hyperparameters are to be tuned, the sampling distribution family, and the sampling distribution parameters.

When working on a new problem, you have to rely on your intuition and experience to define the search space. If you are working on a known problem, then you probably have some information to help you to define a relatively small search space. Tuning is a bit of a guessing game as to what criteria to use to perform the hyperparameter optimization. Generally, you do not want to use too broad of a search space because it is computationally expensive. Alternatively, you don't want to set too narrow a search space so that the globally optimal values are not within that space.

When working with a new problem, there are several common approaches that you could use. For example, you can use a large search space and pay the computational cost of searching the entire space. On the next iteration of the hyperparameter tuning process, you use what you learned about the location of the optimal hyperparameter values to search over a much smaller search space. This approach is good when the computational costs are acceptable or when the problem is not well behaved. Meaning, the function that is being optimized is not smooth and there are sharp cliffs in the value of the cost function. Another approach is to assume that the cost function to be optimized is relatively smooth. This approach is generally a fair assumption for most machine learning models. You specify a large search space and have the hyperparameter tuner perform an initial search. The tuner computes the cost function over the broad search space. As the optimizer runs, you gather information about where the cost function is doing poorly and where it is doing well. This information improves your intuition about what region of the search space that is most likely to contain the optimal hyperparameter values. Then you can shrink the search space to reduce the computational costs.

As `ADSTuner` searches for the optimal values, information about what search criteria are moving toward the optimal value is learned. You use this information to adjust the tuning parameters. Generally, the approaches are to reduce the size of the search space or shift the search space to include values that were not originally in the search space.

The `ADSTuner` intelligently tracks the results of the hyperparameter search. When the tuner is in a terminated state, it is possible to adjust the search space and resume the search. ADSTuner uses the information from its previous search to speed up the resumed search using the new search criteria.

<a id='create'></a>
# Create an `ADSTuner` Object

The `ADSTuner` requires a model class, scoring metric, and a search strategy. In this example, an `GradientBoostingClassifier()` model is used and the $F_1$ weighted average is used as the scoring metric (the metric that we are trying to optimize). The `strategy` parameter defines the search space. A search space requires three pieces of information:

1. The hyperparameter that is to be tuned.
1. The sample distribution family that is used to sample hyperparameter values.
1. The parameters that define the actual sampling distribution.

`ADSTuner` supports the following distribution families:

* `CategoricalDistribution`: Categorical distribution.
* `DiscreteUniformDistribution`: Discrete uniform distribution.
* `IntLogUniformDistribution`: Integer log-uniform distribution.
* `IntUniformDistribution`: Integer uniform distribution.
* `LogUniformDistribution`: Log-uniform distribution.
* `UniformDistribution`: Uniform distribution.

Each distribution family has parameters that define the exact distribution to be sampled from. Generally, these parameters define the range of values to be used in the search space and how to change them is demonstrated in this notebook.

The next cell creates an `ADSTuner` object with the following search space:

|Hyperparameter    |Distribution Family|Distribution Parameters|
|:-                |:-                 |:-                     |
|`max_depth`|Int Uniform            |low=1, high=5      |
|`max_features`   |Categorical        |['sqrt', 'log2']      |
|`n_estimators`       |Integar uniform    |low=50, high=250, step=10  |


In [ ]:
tuner = ADSTuner(model=GradientBoostingClassifier(),
                 scoring=make_scorer(f1_score, average='weighted'), 
                 strategy={'max_depth': IntUniformDistribution(1, 5),
                           'max_features': CategoricalDistribution(['sqrt', 'log2']),
                           'n_estimators': IntUniformDistribution(50, 250, step=10)})

Calling the `search_space()` method with no parameters returns a dictionary that defines the current search space.

In [ ]:
tuner.search_space()

The [iris](https://archive.ics.uci.edu/ml/datasets/iris) dataset is a popular dataset that is used in many pedagogical examples and is used in this notebook to train a toy model.

The next cell loads the iris dataset, and then synchronously tunes a model.

In [ ]:
X,y = load_iris(return_X_y=True)
tuner.tune(X, y, exit_criterion=[NTrials(2)], synchronous=True)

In [ ]:
tuner.status.name

<a id='modify'></a>
# Modify the Search Space

When an `ADSTuner` object is in a terminated state, it is possible to make changes to the search space. There are several ways to put the tuner into a terminated state though it depends on how the tuner was run.

If the tuner was run in a synchronous mode (`tune(synchronous=True)`), then it is terminated when the `tune()` method completes. If `tune()` is still executing, then you can use a keyboard interrupt (control+c) to terminate the tuning operation and put the tuner into a terminated state.

If the tuner was run in an asynchronous mode (`tune(synchronous=False)`), then a call to `terminate()` puts the tuner into a terminated state. When the tuning operation has completed, it is in a terminated state. When in an asynchronous mode, it is possible to halt a tuning process and make changes to the search space. If `resume()` is called, the process does not pick up the changed search space. Changes must be made when the tuner is in a terminated state.

To confirm that the turner is a terminated state, check the `ADSTuner` `status.name` attribute. If it is in a terminated state, it returns the value, 'COMPLETED'.

`ADSTuner` allows these changes to be made to the search space:

- Change the parameters that define the actual sampling distribution for non-categorical distributions.
- Add new hyperparameters.
- Remove existing hyperparameters.

You can't change the distribution family. You can remove it, and then add it back in though the sample distribution family must remain the same. You may want to remove it to limit the search space to see if that hyperparameter is effective. If you decide that it is important, you can be added back in.

<a id='modify_update'></a>
## Update Distribution Hyperparameters

The `search_space()` method of the `ADSTuner` class is used to update the search space. It accepts a dictionary in the same format as the `strategy` parameter in the `tune()` method. By default, the existing search criteria is updated with the changes that are passed in. There is no need to pass in the entire search criteria. This behavior is controlled by the `overwrite` parameter. When `overwrite=False`, the search space is updated. When `overwrite=True`, the search space is replaced by the dictionary that is passed in.

In the next cell, the range of `max_depth` is changed from [1, 5] to [6, 10] and `n_estimators` is changed from [50, 250] to [250, 350].

In [ ]:
tuner.search_space({'max_depth': IntUniformDistribution(6, 10),
                    'n_estimators': IntUniformDistribution(250, 350, step=10)})

The `search_space()` method returns a dictionary of the current search space. The `max_depth` parameter was not passed into `search_space()`, but it is in the returned dictionary. That's because the default behaviour is to update the existing search space with the parameters that have been changed.

The next cell tunes the model using the updated parameters.

In [ ]:
tuner.tune(X, y, exit_criterion=[NTrials(2)], synchronous=True)

The `trials` attribute of `ADSTuner` returns a data frame with trial information. In the next cell, you can see that trials 0 and 1 are from the first time that `tune()` was called. Trials 2 and 3 for the updated search space. You can see that `max_features` is still being tuned because its value is not `NaN`. Also, notice that the values used for `max_depth` changed from [1, 5] to [6, 10] and `n_estimators` changed from [50, 250] to [250, 350].

In [ ]:
tuner.trials[['number', 'value', 'params_max_depth', 'params_max_features', 'params_n_estimators']]

<a id='modify_remove'></a>
## Remove Distribution Hyperparameters

You can remove hyperparameters from the search space. Use the `search_space()` method with the `overwrite` parameter set to `True`. The dictionary that is passed in defines the search space.

When removing hyperparameters, you can change the sampling distribution parameters of the already existing hyperparameters. This is similar to  the <a href='modify_update'>update distribution parameters</a> operation.

In the next cell, the `max_features` hyperparameter is removed by not including it in the dictionary. The `max_depth` changed from [6, 10] to [10, 12] and `n_estimators` changed from [250, 350] to [350, 500]. Notice that the `overwrite` parameter is set to `True` so the search space is overwritten with the new strategy. The output of the cell gives a dictionary. You can see that `max_features` is not listed as a tuning parameter and that `max_depth` and `n_estimators` have been updated.

In [ ]:
tuner.search_space({'max_depth': IntUniformDistribution(10, 12),
                    'n_estimators': IntUniformDistribution(350, 500, step=10)}, 
                   overwrite=True)

In the next cell, a new tuning operation is performed and the tuning trials are printed. The new trials are numbers 4 and 5. The `NaN` in the `param_max_features` column indicates that this hyperparameter was not being tuned. You can also see that the sampling distribution parameters for `max_depth` are in the updated range of [10, 12] and [350, 500] for `n_estimators`.

In [ ]:
tuner.tune(X, y, exit_criterion=[NTrials(2)], synchronous=True)
tuner.trials[['number', 'value', 'params_max_depth', 'params_max_features', 'params_n_estimators']]

<a id='modify_add'></a>
## Add Distribution Hyperparameters

There are two ways that a hyperparameter can be added to search space. The first is to follow the steps that were outlined in  <a href='modify_update'>Update Distribution Hyperparameters</a> using a call to the `search_space()` method with `overwrite=False,` and passing in a dictionary of the hyperparameters to add.

The second approach is similar to what was described in <a href='modify_remove'>Remove Distribution Hyperparameters</a>. However, instead of removing hyperparameters, they are added. The parameter `overwrite` in the `search_space()` method must be set to `True`.

In both cases, you can modify existing hyperparameters while adding new hyperparameters.

If a hyperparameter is removed, it can only be added back in if the sample distribution family is the same.

In the next cell, the `learning_rate` hyperparameter is added to the existing search space. The `search_space()` method returns a dictionary of the search space.

In [ ]:
tuner.search_space({'learning_rate': LogUniformDistribution(0.001, 0.6)}, overwrite=False)

In the next cell, a new tuning operation is performed and the tuning trials are printed. The new trials are numbers 6 and 7. In the `params_learning_rate` column the `NaN` in trials 0 to 5 indicate that it was not being tuned. However, in trials 6 and 7 there are floating-point numbers indicating that `learning_rate` is being tuned.

In [ ]:
tuner.tune(X, y, exit_criterion=[NTrials(2)], synchronous=True)
tuner.trials[['number', 'value', 'params_max_depth', 'params_max_features', 'params_n_estimators', 'params_learning_rate']]

<a id="ref"></a>
## References
- [ADS Library Documentation](https://docs.cloud.oracle.com/en-us/iaas/tools/ads-sdk/latest/index.html)
- The [iris](https://archive.ics.uci.edu/ml/datasets/iris) dataset.
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)